## Load in data

In [4]:
import pandas as pd

monday_data = pd.read_csv('data/Monday-WorkingHours.pcap_ISCX.csv')
tuesday_data = pd.read_csv('data/Tuesday-WorkingHours.pcap_ISCX.csv')
wednesday_data = pd.read_csv('data/Wednesday-workingHours.pcap_ISCX.csv')
thursday_web_attacks_data = pd.read_csv('data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
thursday_infiltration_data = pd.read_csv('data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
friday_morning_data = pd.read_csv('data/Friday-WorkingHours-Morning.pcap_ISCX.csv')
friday_port_scan_data = pd.read_csv('data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
friday_ddos_data = pd.read_csv('data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [6]:
full_data = pd.concat([monday_data, tuesday_data, wednesday_data,
                       thursday_web_attacks_data, thursday_infiltration_data,
                       friday_morning_data, friday_port_scan_data, friday_ddos_data], ignore_index=True)

print(full_data.shape)
full_data.head()

(2830743, 79)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [9]:
full_data.columns = full_data.columns.str.strip()

In [11]:
full_data.Label.value_counts()

Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64